# KNN Classifier - Penguin Classification
## Diseño de una solución orientada a objetos para clasificar pingüinos
Objetivo: Clasificar pingüinos usando KNN sin bibliotecas externas como scikit-learn.

In [ ]:
# ===============================
# 1. IMPORTACIONES NECESARIAS
# ===============================
import pandas as pd
import numpy as np
from abc import ABC, abstractmethod
from collections import Counter

In [ ]:
# ===============================
# 2. CLASE ABSTRACTA BASE
# ===============================
class KNNBase(ABC):
    @abstractmethod
    def fit(self, X, y):
        pass

    @abstractmethod
    def distance(self, p1, p2):
        pass

    @abstractmethod
    def predict(self, X_new, k=3):
        pass

In [ ]:
# =============================================
# 3. CLASE CONCRETA: KNNClassifier
# =============================================
class KNNClassifier(KNNBase):
    def __init__(self):
        self.__X_train = None
        self.__y_train = None

    def fit(self, X, y):
        self.__X_train = X
        self.__y_train = y

    def distance(self, p1, p2):
        return np.linalg.norm(p1 - p2)

    def predict(self, X_new, k=3):
        predictions = []
        for x in X_new:
            distances = [self.distance(x, x_train) for x_train in self.__X_train]
            k_indices = np.argsort(distances)[:k]
            k_labels = [self.__y_train[i] for i in k_indices]
            most_common = Counter(k_labels).most_common(1)[0][0]
            predictions.append(most_common)
        return predictions

    def __repr__(self):
        return f"KNNClassifier(model trained: {self.__X_train is not None})"

    def __eq__(self, other):
        return np.array_equal(self.__X_train, other.__X_train) and np.array_equal(self.__y_train, other.__y_train)

    def __add__(self, other):
        new_X = np.vstack((self.__X_train, other.__X_train))
        new_y = np.hstack((self.__y_train, other.__y_train))
        combined = KNNClassifier()
        combined.fit(new_X, new_y)
        return combined

In [ ]:
# ===============================
# 4. CARGAR Y PREPARAR EL DATASET
# ===============================
df = pd.read_csv("pinguins.csv")
df = df.dropna()
df['Species'] = df['Species'].str.extract(r'(Adelie|Chinstrap|Gentoo)', expand=False)
X = df[['Culmen Length (mm)', 'Culmen Depth (mm)', 'Flipper Length (mm)', 'Body Mass (g)']].values
y = df['Species'].values

In [ ]:
# ===============================
# 5. ENTRENAR Y PROBAR EL MODELO
# ===============================
knn = KNNClassifier()
knn.fit(X, y)
X_test = X[:10]
y_real = y[:10]
resultados = {
    "k=1": knn.predict(X_test, k=1),
    "k=3": knn.predict(X_test, k=3),
    "k=5": knn.predict(X_test, k=5),
    "Real": y_real
}
pd.DataFrame(resultados)